# This is will focus on getting inputs in different formats

In [50]:
import numpy as np
import pandas as pd
import gamspy as gp

In [51]:
m = gp.Container()

## Inputs as dataframes 

In [52]:
# Create pd dataframe from dictionary
data = {'city': ['New York', 'Chicago', 'Topeka'],
        'supply': [300, 400, 500]}
df = pd.DataFrame(data)
df.set_index('city', inplace=True)
display(df)

# create sets and parameters using data frame instead of manually writing them down
cities = gp.Set(m, 'cities',records=df.index)  # j
supply = gp.Parameter(m, 'supply', domain=[cities])  # s
supply.setRecords(df['supply'])
display(cities.records, supply.records)

,supply
city,
New York,300
Chicago,400
Topeka,500


,uni,element_text
0,New York,
1,Chicago,
2,Topeka,


,cities,value
0,New York,300.0
1,Chicago,400.0
2,Topeka,500.0


## Inputs as 2d matrices  

In [53]:
# reset the container
m = gp.Container()
cities = [i+1 for i in range(6)]
# distances to travel between 6 different locations (diagonal is 0)
dist = np.array([
    [ 0,      10,     20,     30,    30,    20],
    [10,       0,     25,     35,    20,    10],
    [20,      25,      0,     15,    30,    20],
    [30,      35,     15,      0,    15,    25],
    [30,      20,     30,     15,     0,    14],
    [20,      10,     20,     25,    14,     0]])
travelTime = pd.DataFrame(data=dist,index=cities, columns=cities)
display(travelTime)

,1,2,3,4,5,6
1,0,10,20,30,30,20
2,10,0,25,35,20,10
3,20,25,0,15,30,20
4,30,35,15,0,15,25
5,30,20,30,15,0,14
6,20,10,20,25,14,0


In [54]:
# read from dataframe and populate domain sets at same time
i = gp.Set(m,'i',description='city')    # start
j = gp.Set(m,'j',description='city')    # destination 
tTime = gp.Parameter(
    m,
    'travelTime',
    [i,j],
    domain_forwarding=True,         # automatically create the sets i and j by reading the row and column headers from the travelTime DataFrame you're about to load
    description="time from i to j",
    records=travelTime,
    uels_on_axes=True
    )
display(i.records,tTime.records)

,uni,element_text
0,1,
1,2,
2,3,
3,4,
4,5,
5,6,


,i,j,value
0,1,1,0.0
1,1,2,10.0
2,1,3,20.0
3,1,4,30.0
4,1,5,30.0
5,1,6,20.0
6,2,1,10.0
7,2,2,0.0
8,2,3,25.0
9,2,4,35.0


In [55]:
# create 2D "sparse" data
arcs = travelTime[travelTime > 0].stack().reset_index()
arcs.columns = ['fr','to','val']

# import 2D dataframe as set
fr = gp.Set(m,'fr')
to = gp.Set(m,'to')
a = gp.Set(m,'a',domain=[fr,to],domain_forwarding=True,records=arcs[['fr','to']])

# import 2D dataframe as parameter
e = gp.Parameter(m,'e',domain=[fr,to],records=arcs[['fr','to','val']])
display(e.records)

,fr,to,value
0,1,2,10.0
1,1,3,20.0
2,1,4,30.0
3,1,5,30.0
4,1,6,20.0
5,2,1,10.0
6,2,3,25.0
7,2,4,35.0
8,2,5,20.0
9,2,6,10.0


In [56]:
# reset container
m = gp.Container() 

cities = [i+1 for i in range(6)]
dist = np.array([
    [ 0,      10,     20,     30,    30,    20],
    [10,       0,     25,     35,    20,    10],
    [20,      25,      0,     15,    30,    20],
    [30,      35,     15,      0,    15,    25],
    [30,      20,     30,     15,     0,    14],
    [20,      10,     20,     25,    14,     0]])
travelTime = pd.DataFrame(data=dist,index=cities, columns=cities)
display(travelTime)

,1,2,3,4,5,6
1,0,10,20,30,30,20
2,10,0,25,35,20,10
3,20,25,0,15,30,20
4,30,35,15,0,15,25
5,30,20,30,15,0,14
6,20,10,20,25,14,0


In [57]:
# already have cities defined
i = gp.Set(m,'i',description='city',records=cities)
j = gp.Set(m,'j',description='city',records=cities)

# array shape matches the domain so this works fine
tTime = gp.Parameter(m,'travelTime',[i,j],description="time from i to j",records=dist)
display(tTime.records,tTime.pivot())
# notice how start == dest isn't there

,i,j,value
0,1,2,10.0
1,1,3,20.0
2,1,4,30.0
3,1,5,30.0
4,1,6,20.0
5,2,1,10.0
6,2,3,25.0
7,2,4,35.0
8,2,5,20.0
9,2,6,10.0


,1,2,3,4,5,6
1,0.0,10.0,20.0,30.0,30.0,20.0
2,10.0,0.0,25.0,35.0,20.0,10.0
3,20.0,25.0,0.0,15.0,30.0,20.0
4,30.0,35.0,15.0,0.0,15.0,25.0
5,30.0,20.0,30.0,15.0,0.0,14.0
6,20.0,10.0,20.0,25.0,14.0,0.0


In [58]:
# Flatten the data (and add extra column to dataframe)
tTdf = travelTime.stack().reset_index()
tTdf.columns = ['i','j','level']
tTdf['marginal'] = np.ones([len(tTdf),1])
display(tTdf)

,i,j,level,marginal
0,1,1,0,1.0
1,1,2,10,1.0
2,1,3,20,1.0
3,1,4,30,1.0
4,1,5,30,1.0
5,1,6,20,1.0
6,2,1,10,1.0
7,2,2,0,1.0
8,2,3,25,1.0
9,2,4,35,1.0


In [59]:
tVar = gp.Variable(m,'tVar','positive',[i,j],description="time from i to j",records=travelTime,uels_on_axes=True)
display(tVar.records)

,i,j,level,marginal,lower,upper,scale
0,1,1,0.0,0.0,0.0,inf,1.0
1,1,2,10.0,0.0,0.0,inf,1.0
2,1,3,20.0,0.0,0.0,inf,1.0
3,1,4,30.0,0.0,0.0,inf,1.0
4,1,5,30.0,0.0,0.0,inf,1.0
5,1,6,20.0,0.0,0.0,inf,1.0
6,2,1,10.0,0.0,0.0,inf,1.0
7,2,2,0.0,0.0,0.0,inf,1.0
8,2,3,25.0,0.0,0.0,inf,1.0
9,2,4,35.0,0.0,0.0,inf,1.0


## Write data to a gdx file

In [60]:
i = gp.Set(m,'a')
j = gp.Set(m,'b')
dist = pd.DataFrame(
    [
        ("seattle", "new-york", 2.5),
        ("seattle", "chicago", 1.7),
        ("seattle", "topeka", 1.8),
        ("san-diego", "new-york", 2.5),
        ("san-diego", "chicago", 1.8),
        ("san-diego", "topeka", 1.4),
    ],
    columns=["from", "to", "thousand_miles"],
)
display(dist)

,from,to,thousand_miles
0,seattle,new-york,2.5
1,seattle,chicago,1.7
2,seattle,topeka,1.8
3,san-diego,new-york,2.5
4,san-diego,chicago,1.8
5,san-diego,topeka,1.4


In [61]:
d = gp.Parameter(m,'d', domain=[i,j], records = dist,
    description='transport in thousands of miles', domain_forwarding=True);

# wite the container to a gdx file
m.write('travel.gdx',compress=False)

## Write data to a csv

In [62]:
# Extract dataframe and write to csv
d.records.to_csv('travel.csv',index=False)

## Write a an excel file

In [63]:
excel_file = 'travel.xlsx'
writer = pd.ExcelWriter(excel_file)
# Convert the dataframe to an XlsxWriter Excel object.
# Can write multiple sheets using this routine
i.records.to_excel(writer, sheet_name='Sheet1')
d.records.to_excel(writer, sheet_name='Sheet2')
# remove index column 
d.records.to_excel(writer, sheet_name='Sheet3',index=False)
# Close the Pandas Excel writer and output the Excel file.
writer.close()

## Loading from a gdx file

In [64]:
m = gp.Container(load_from='travel.gdx')
for name, symbol in m:
    print(f"Name: {name}, records: {symbol.records}")

Name: i, records:   uni element_text
0   1             
1   2             
2   3             
3   4             
4   5             
5   6             
Name: j, records:   uni element_text
0   1             
1   2             
2   3             
3   4             
4   5             
5   6             
Name: travelTime, records:     i  j  value
0   1  2   10.0
1   1  3   20.0
2   1  4   30.0
3   1  5   30.0
4   1  6   20.0
5   2  1   10.0
6   2  3   25.0
7   2  4   35.0
8   2  5   20.0
9   2  6   10.0
10  3  1   20.0
11  3  2   25.0
12  3  4   15.0
13  3  5   30.0
14  3  6   20.0
15  4  1   30.0
16  4  2   35.0
17  4  3   15.0
18  4  5   15.0
19  4  6   25.0
20  5  1   30.0
21  5  2   20.0
22  5  3   30.0
23  5  4   15.0
24  5  6   14.0
25  6  1   20.0
26  6  2   10.0
27  6  3   20.0
28  6  4   25.0
29  6  5   14.0
Name: tVar, records:     i  j  level  marginal  lower  upper  scale
0   1  1    0.0       0.0    0.0    inf    1.0
1   1  2   10.0       0.0    0.0    inf    1.0
2   1  3   20

## Read from excel

In [65]:
xls_file = pd.ExcelFile("NFL.xlsx")

player = xls_file.parse('Sheet1',usecols=['Player'],dtype={'Player':str})

cols = {'Player': str,'Salary': float}
salary = xls_file.parse('Sheet1',usecols=cols.keys(),dtype=cols)

cols = {'Player': str,'PFF': float}
PFF = xls_file.parse('Sheet2',usecols=cols.keys(),dtype=cols)

cols = {'Player': str,'Pos_Numbered': int}
Pos_Numbered = xls_file.parse('Sheet4',usecols=cols.keys(),dtype=cols)

m = gp.Container() 

gp.Set(m,"player", ['*'], records=player)
gp.Parameter(m,"salary", ['player'], records=salary)
gp.Parameter(m,"PFF", ['player'], records=PFF)
gp.Parameter(m,"Pos_Numbered", ['player'], records=Pos_Numbered)

m.write('NFL.gdx', compress=False)

## Extra: read from csv with multiple indices

In [66]:
m = gp.Container()
df = pd.read_csv('openhubs.csv')
#, index_col = ["Provider.PIN"])
df.rename(columns={"Provider.PIN": "provider","Hub.ID": "hub", "HERC.Number": "herc","FIPS": "fips","Org.Name": "region"},inplace=True)
provider = gp.Set(m,'provider',records=df.provider.unique())
region = gp.Set(m,'region',records=df.region.unique())
# could do other 1-D sets similarly
sname = gp.Set(m,'sname',[provider,region],records=df[['provider','region']].drop_duplicates())
df = df.set_index(["provider"])
hubdata = gp.Parameter(m,'hubdata',[provider,'*'],records=df[['county.pop','ULT.capacity','ULT.flag','p.ulo','m.ulo','p.einv','m.einv','cap.cat']].stack())
df = df.reset_index().set_index(["provider","hub","herc","county","fips","region"])
# from2dim(df) = df.stack().reset_index()
data = gp.Parameter(m,'data',[provider,'*','*','*','*',region,'*'],records=df.stack())
display(provider.records)
display(sname.pivot(fill_value=""))
display(data.records,data.pivot())
display(hubdata.pivot())

,uni,element_text
0,778094,
1,778095,
2,778096,
3,778097,
4,778098,


,Region 1 Hub - Eau Claire,Region 2 Hub - Marshfield,Region 3 Hub - Green Bay,Region 5 Hub - Madison,Region 7 Hub - Milwaukee
778094,True,,,,
778095,,True,,,
778096,,,True,,
778097,,,,True,
778098,,,,,True


,provider_0,uni_1,uni_2,uni_3,uni_4,region_5,uni_6,value
0,778094,778094,1,Eau Claire,55035,Region 1 Hub - Eau Claire,county.pop,103301.0
1,778094,778094,1,Eau Claire,55035,Region 1 Hub - Eau Claire,ULT.capacity,200000.0
2,778094,778094,1,Eau Claire,55035,Region 1 Hub - Eau Claire,ULT.flag,1.0
3,778094,778094,1,Eau Claire,55035,Region 1 Hub - Eau Claire,p.ulo,300.0
4,778094,778094,1,Eau Claire,55035,Region 1 Hub - Eau Claire,p.einv,300.0
5,778094,778094,1,Eau Claire,55035,Region 1 Hub - Eau Claire,cap.cat,1.0
6,778095,778095,2,Wood,55141,Region 2 Hub - Marshfield,county.pop,73975.0
7,778095,778095,2,Wood,55141,Region 2 Hub - Marshfield,ULT.capacity,150000.0
8,778095,778095,2,Wood,55141,Region 2 Hub - Marshfield,ULT.flag,1.0
9,778095,778095,2,Wood,55141,Region 2 Hub - Marshfield,p.ulo,210.0


,,,,,,county.pop,ULT.capacity,ULT.flag,p.ulo,p.einv,cap.cat,m.ulo,m.einv
778094,778094,1,Eau Claire,55035,Region 1 Hub - Eau Claire,103301.0,200000.0,1.0,300.0,300.0,1.0,0.0,0.0
778095,778095,2,Wood,55141,Region 2 Hub - Marshfield,73975.0,150000.0,1.0,210.0,210.0,1.0,0.0,0.0
778096,778096,3,Brown,55009,Region 3 Hub - Green Bay,261232.0,35100.0,1.0,210.0,210.0,1.0,0.0,0.0
778097,778097,5,Dane,55025,Region 5 Hub - Madison,533360.0,195000.0,1.0,420.0,420.0,1.0,500.0,500.0
778098,778098,7,Milwaukee,55079,Region 7 Hub - Milwaukee,950358.0,150000.0,1.0,420.0,420.0,1.0,0.0,0.0


,county.pop,ULT.capacity,ULT.flag,p.ulo,p.einv,cap.cat,m.ulo,m.einv
778094,103301.0,200000.0,1.0,300.0,300.0,1.0,0.0,0.0
778095,73975.0,150000.0,1.0,210.0,210.0,1.0,0.0,0.0
778096,261232.0,35100.0,1.0,210.0,210.0,1.0,0.0,0.0
778097,533360.0,195000.0,1.0,420.0,420.0,1.0,500.0,500.0
778098,950358.0,150000.0,1.0,420.0,420.0,1.0,0.0,0.0
